In [1]:
import pandas as pd 
import csv
from geopy.geocoders import Bing
import geopandas as gpd


In [6]:
import folium
import pandas as pd
import math
from matplotlib import cm, colors
from IPython.display import HTML
import geopandas as gpd

# Load US states shapefile
us_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Filter US states by name
us_states = us_states[us_states['name'].isin([
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
    'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
    'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
    'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
    'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
    'New Jersey', 'New Mexico', 'New York', 'North Carolina',
    'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
    'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
    'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming'
])]

# Save the filtered data as a GeoJSON file
us_states.to_file("us-states.geojson", driver='GeoJSON')

# Load the updated clean datasets
df_1 = pd.read_csv("Heart_36_Col.csv")
df_2 = pd.read_csv("Heart_42_Col.csv")

# Function to create choropleth map using Folium
def create_choropleth_map(df):
    m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

    # Count cases for each state
    state_counts = df['State'].value_counts().to_dict()
    state_coordinates = df.groupby('State')[['Latitude', 'Longitude']].mean().to_dict('index')

    # Get unique case counts
    unique_counts = sorted(set(state_counts.values()), reverse=True)

    # Generate a continuous colormap (reversed)
    colormap = cm.get_cmap('tab20c', len(unique_counts))
    count_to_color = {count: colors.rgb2hex(colormap(i / len(unique_counts))) for i, count in enumerate(unique_counts)}

    # Add markers to the map based on case counts
    for state, count in state_counts.items():
        lat = state_coordinates[state]['Latitude']
        lon = state_coordinates[state]['Longitude']
        color = count_to_color[count]

        # Set marker size based on square root of case count, exaggerated
        marker_size = math.sqrt(count) * 0.2

        # Add marker to the map
        popup_text = f'<div style="width:250px; height:40px;"><b>State:</b> {state}<br><b>Cases:</b> {count}</div>'
        folium.CircleMarker(location=[lat, lon], radius=marker_size, color=color, fill=True, fill_opacity=0.6,
                            popup=popup_text).add_to(m)

    # Create a scrollable legend
    legend_html = '''
        <div style="position: fixed; 
                    bottom: 20px; right: 10px; width: 200px; height: 250px; 
                    background-color: white; border-radius: 5px; z-index:9999;
                    overflow-y: scroll; padding: 10px; font-size: 10px;">
            <b>Legend</b><br>
    '''
    for count in unique_counts:
        legend_html += f'<i style="background:{count_to_color[count]}; width: 10px; height: 10px; display:inline-block"></i> Cases: {count}<br>'
    legend_html += '</div>'
    
    # Add legend to map
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

# Call the function to create the choropleth map
choropleth_map_36 = create_choropleth_map(df_1)
choropleth_map_42 = create_choropleth_map(df_2)

# Save the maps to HTML files
choropleth_map_36.save('choropleth_map_36.html')
choropleth_map_42.save('choropleth_map_42.html')

# Convert the maps to HTML and display them
html_map_36 = choropleth_map_36._repr_html_()
html_map_42 = choropleth_map_42._repr_html_()

# Display the maps
display(HTML(html_map_36))
display(HTML(html_map_42))


C:\Users\faran\AppData\Local\Temp\ipykernel_27120\3009266646.py:9: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  us_states = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\faran\AppData\Local\Temp\ipykernel_27120\3009266646.py:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('tab20c', len(unique_counts))
C:\Users\faran\AppData\Local\Temp\ipykernel_27120\3009266646.py:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = c